In [1]:
import multiprocessing as mp

In [4]:
def worker_main_test_ucr_p1(train_file, test_file, wd_arr, wl_arr, out_q):
    try:
        print("start worker")
        wd_num = len(wd_arr)
        wl_num = len(wl_arr)
    
        bopf = BagOfPatternFeature()
        bopf.load_dataset(train_file, fmt="file")
        bopf.cumsum()
    
        bopf_t = BagOfPatternFeature()
        bopf_t.load_dataset(test_file, fmt="file")
        bopf_t.cumsum()
    
        output_dict = defaultdict(list)
    
        for i in range(wd_num):
            wd = wd_arr[i]
            for j in range(wl_num):
                wl = wl_arr[j]
                bopf.bop(wd, wl, verbose=False)
                bopf.adjust_label_set()
                bopf.anova(verbose=False)
                bopf.anova_sort()
                bopf.sort_trim_arr(verbose=False)
                bopf.crossVL(verbose=False)
                output_dict["bop_features"].append(bopf.crossL[:bopf.c*bopf.best_idx])
                output_dict["bop_fea_num"].append(bopf.best_idx)
                output_dict["bop_cv_acc"].append(bopf.best_score)
                output_dict["bop_feature_index"].append(bopf.sort_index[:bopf.best_idx])
            
                bopf.crossVL2()
                output_dict["bop_features2"].append(bopf.crossL2[:bopf.c*bopf.best2_idx])
                output_dict["bop_fea_num2"].append(bopf.best2_idx)
                output_dict["bop_cv_acc2"].append(bopf.best2_score)
                output_dict["bop_feature_index2"].append(bopf.sort_index[:bopf.best2_idx])
            
                output_dict["bop_wd"].append(wd)
                output_dict["bop_wl"].append(wl)
        
        out_q.put((wl_arr[0], wl_arr[-1], output_dict))
    except:
        print("worker failed")
    finally:
        print("done")
    
    

def main_test_ucr_p1_multiprocess(train_file, test_file, wd_arr, wl_arr):
    m = mp.Manager()
    result_queue = m.Queue()
    
    N = 1 + (len(wl_arr) // n_process)
    
    for k in range(n_process):
        i = k*N
        j = (k+1)*N
        if j > len(wl_arr):
            j = len(wl_arr)
        wl_sub_arr = wl_arr[i:j]
        jobs.append(mp.Process(target=worker_main_test_ucr_p1, 
                               args=(train_file, test_file, wd_arr, wl_sub_arr, result_queue)))
        jobs[-1].start()
    
    return bopf, bopf_t, output_dict

def worker_main_test_ucr_p2(i, j, index1, index2, output_dict, bopf, test_file, out_q):
    
    bopf_t = BagOfPatternFeature()
    bopf_t.load_dataset(test_file, fmt="file")
    bopf_t.cumsum()
    
    best_centroid = -1
    best_tf_idf = -1
    rbest_centroid = -np.inf
    rbest_tf_idf = -np.inf
    for k in range(i, j):
        s_index = index1[k]
        wd = output_dict["bop_wd"][s_index]
        wl = output_dict["bop_wl"][s_index]
        bopf_t.bop(wd, wl, verbose=False)
        test_bop_sort = sort_trim_arr(bopf_t.train_bop, output_dict["bop_feature_index"][s_index], 
                                  bopf_t.m, output_dict["bop_fea_num"][s_index])
        predicted_label = classify(test_bop_sort, output_dict["bop_features"][s_index], bopf.tlabel, 
                               bopf_t.m, bopf.c, output_dict["bop_fea_num"][s_index])
        real_label = np.array(bopf_t.labels)
        count = 0
        for lbl_i in range(len(real_label)):
            if predicted_label[lbl_i] == real_label[lbl_i]:
                count += 1
        acc = count / len(real_label)
        if acc > rbest_centroid:
            rbest_centroid = acc
            best_centroid = i
            
    for k in range(i, j):
        s_index = index2[k]
        wd = output_dict["bop_wd"][s_index]
        wl = output_dict["bop_wl"][s_index]
        bopf_t.bop(wd, wl, verbose=False)
        test_bop_sort = sort_trim_arr(bopf_t.train_bop, output_dict["bop_feature_index2"][s_index], 
                              bopf_t.m, output_dict["bop_fea_num2"][s_index])
        predicted_label = classify2(test_bop_sort, output_dict["bop_features2"][s_index], bopf.tlabel, 
                           bopf_t.m, bopf.c, output_dict["bop_fea_num2"][s_index])
        real_label = np.array(bopf_t.labels)
        count = 0
        for lbl_i in range(len(real_label)):
            if predicted_label[lbl_i] == real_label[lbl_i]:
                count += 1
        acc = count / len(real_label)
        if acc > rbest_tf_idf:
            rbest_tf_idf = acc
            best_tf_idf = i
            
    out_q.put((rbest_centroid, best_centroid, rbest_tf_idf, best_tf_idf))
        
def main_test_ucr_p2_multiprocess(train_file, test_file, output_dict, top_n, n_process):
    
    m = mp.Manager()
    result_queue = m.Queue()
    
    bopf = BagOfPatternFeature()
    bopf.load_dataset(train_file, fmt="file")
    bopf.cumsum()
    
    index1 = np.argsort(output_dict["bop_cv_acc"])[::-1]
    index2 = np.argsort(output_dict["bop_cv_acc2"])[::-1]
    
    N = 1 + (top_n // n_process)
    jobs = []
    for k in range(n_process):
        i = k*N
        j = (k+1)*N
        if j > top_n:
            j = top_n
        jobs.append(mp.Process(target=worker_main_test_ucr_p2, 
                               args=(i, j, index1, index2, output_dict, bopf, test_file, result_queue)))
        jobs[-1].start()
    
    for p in jobs:
        p.join()
        
    best_centroid = -1
    best_tf_idf = -1
    rbest_centroid = -np.inf
    rbest_tf_idf = -np.inf
    
    num_res = result_queue.size()
    while num_ress > 0:
        rbest_centroid_i, best_centroid_i, rbest_tf_idf_i, best_tf_idf_i = result_queue.get()
        num_ress -= 1
        if rbest_centroid_i > rbest_centroid:
            best_centroid = best_centroid_i
        if rbest_tf_idf_i > rbest_tf_idf:
            best_tf_idf = best_tf_idf_i

    s_index = index1[best_centroid]
    print("classify with best centroid and wd:", output_dict["bop_wd"][s_index], 
          ", wl:", output_dict["bop_wl"][s_index], 
          "-> cv_acc:", round(output_dict["bop_cv_acc"][s_index], 3),
          ", acc:", round(rbest_centroid, 3))
    
    s_index = index2[best_tf_idf]
    print("classify with best tf-idf and wd:", output_dict["bop_wd"][s_index], 
          ", wl:", output_dict["bop_wl"][s_index], 
          "-> cv_acc:", round(output_dict["bop_cv_acc2"][s_index], 3),
          ", acc:", round(rbest_tf_idf, 3))

In [ ]:
%%time
train_file = "D:/tesis/UCRArchive_2018/Adiac/Adiac_TRAIN.tsv"
test_file = "D:/tesis/UCRArchive_2018/Adiac/Adiac_TEST.tsv"
bopf2, bopf_t2, output_dict2 = main_test_ucr_p1(train_file, test_file)

In [1]:
import sys
import os
from collections import defaultdict
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("./Untitled.ipynb"), '..')))
import numpy as np
from src.bopf.bopf import BagOfPatternFeature
from src.bopf.classifier import classify, classify2
from src.utils import sort_trim_arr

c:\programdata\miniconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\programdata\miniconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\programdata\miniconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [4]:
bopf = BagOfPatternFeature(special_character=True)
path = "D:/tesis/tesis/data/plasticc_sub_dataset/"
bopf.load_dataset(path, fmt="npy", set_type="train", n1=500, n2=500, c=4)
print("train size:", bopf.m)
bopf.cumsum()
wd = 7 
wl = 0.325
bopf.bop(wd, wl)
bopf.adjust_label_set()
bopf.anova()
bopf.anova_sort()
bopf.sort_trim_arr()
bopf.crossVL(verbose=False)
print("best accuracy of CV: ", bopf.best_score, ", using the top", bopf.best_idx, "features according to ANOVA F")
bopf.crossVL2()
print("best accuracy of CV2: ", bopf.best2_score, ", using the top", bopf.best2_idx, "features according to ANOVA F")


# load the test set
bopf_t = BagOfPatternFeature(special_character=True)
bopf_t.load_dataset(path, fmt="npy", set_type="test", n1=500, n2=500, c=4)

# compute CUMSUM matrix for test set
bopf_t.cumsum()

# compute bag-of-pattern representation for test set
bopf_t.bop(wd, wl)

# sort and trim the bag-of-pattern representation based on the centroid best configuration
test_bop_sort = sort_trim_arr(bopf_t.train_bop, bopf.sort_index[:bopf.best_idx], 
                                  bopf_t.m, bopf.best_idx)
# classify to get predicted labels
predicted_label = classify(test_bop_sort, bopf.crossL[:bopf. c*bopf.best_idx], bopf.tlabel, 
                               bopf_t.m, bopf.c, bopf.best_idx)

# get real labels of test set
real_label = np.array(bopf_t.labels)

# compute accuracy
count = 0
for i in range(len(real_label)):
    if predicted_label[i] == real_label[i]:
        count += 1

acc = count / bopf_t.m
print("accuracy of classify using wd:", wd, ", wl:", wl, ", and best config for centroids is:", acc)

### repeat for tf-idf

# sort and trim the bag-of-pattern representation based on the centroid best configuration
test_bop_sort2 = sort_trim_arr(bopf_t.train_bop, bopf.sort_index[:bopf.best2_idx], 
                                  bopf_t.m, bopf.best2_idx)
# classify to get predicted labels
predicted_label2 = classify2(test_bop_sort2, bopf.crossL2[:bopf.c*bopf.best2_idx], bopf.tlabel, 
                               bopf_t.m, bopf.c, bopf.best2_idx)

# compute accuracy
count = 0
for i in range(len(real_label)):
    if predicted_label2[i] == real_label[i]:
        count += 1
        
acc2 = count / len(real_label)
print("accuracy of classify using wd:", wd, ", wl:", wl, ", and best config for tf-idf is:", acc2)

train size: 500
TOTAL DE SEGMENTOS VACIOS:  41441
best accuracy of CV:  0.35 , using the top 118 features according to ANOVA F
best accuracy of CV2:  0.834 , using the top 790 features according to ANOVA F
TOTAL DE SEGMENTOS VACIOS:  43393
accuracy of classify using wd: 7 , wl: 0.325 , and best config for centroids is: 0.226
accuracy of classify using wd: 7 , wl: 0.325 , and best config for tf-idf is: 0.168


In [ ]:
def main_test_plasticc_p1(n1, n2, c, wd_arr, wl_arr):
    wd_num = len(wd_arr)
    wl_num = len(wl_arr)
    
    
    
    bopf = BagOfPatternFeature(special_character=True)
    path = "D:/tesis/tesis/data/plasticc_sub_dataset/"
    bopf.load_dataset(path, fmt="npy", set_type="train", n1=n1, c=c)
    bopf.cumsum()
    
    bopf_t = BagOfPatternFeature(special_character=True)
    bopf_t.load_dataset(path, fmt="npy", set_type="test", n1=n2, c=c)
    bopf_t.cumsum()
    
    output_dict = defaultdict(list)
    
    for i in range(wd_num):
        wd = wd_arr[i]
        for j in range(wl_num):
            wl = wl_arr[j]
            print("wd: {}".format(wd) + ", wl: {}".format(wl), end="\r")
            bopf.bop(wd, wl, verbose=False)
            bopf.adjust_label_set()
            bopf.anova(verbose=False)
            bopf.anova_sort()
            bopf.sort_trim_arr(verbose=False)
            
            bopf.crossVL(verbose=False)
            output_dict["bop_features"].append(bopf.crossL[:bopf.c*bopf.best_idx])
            output_dict["bop_fea_num"].append(bopf.best_idx)
            output_dict["bop_cv_acc"].append(bopf.best_score)
            output_dict["bop_feature_index"].append(bopf.sort_index[:bopf.best_idx])
            
            bopf.crossVL2()
            output_dict["bop_features2"].append(bopf.crossL2[:bopf.c*bopf.best2_idx])
            output_dict["bop_fea_num2"].append(bopf.best2_idx)
            output_dict["bop_cv_acc2"].append(bopf.best2_score)
            output_dict["bop_feature_index2"].append(bopf.sort_index[:bopf.best2_idx])
            
            output_dict["bop_wd"].append(wd)
            output_dict["bop_wl"].append(wl)
            
    print("wd: {}".format(wd) + ", wl: {0:.2f}".format(wl))
    return bopf, bopf_t, output_dict
    
def main_test_plasticc_p2(bopf, bopf_t, output_dict, top_n):
    index1 = np.argsort(output_dict["bop_cv_acc"])[::-1]
    index2 = np.argsort(output_dict["bop_cv_acc2"])[::-1]

    best_centroid = -1
    best_tf_idf = -1
    rbest_centroid = -np.inf
    rbest_tf_idf = -np.inf
    for i in range(top_n):
        print(i, end="\r")
        s_index = index1[i]
        wd = output_dict["bop_wd"][s_index]
        wl = output_dict["bop_wl"][s_index]
        bopf_t.bop(wd, wl, verbose=False)
        test_bop_sort = sort_trim_arr(bopf_t.train_bop, output_dict["bop_feature_index"][s_index], 
                                  bopf_t.m, output_dict["bop_fea_num"][s_index])
        predicted_label = classify(test_bop_sort, output_dict["bop_features"][s_index], bopf.tlabel, 
                               bopf_t.m, bopf.c, output_dict["bop_fea_num"][s_index])
        real_label = np.array(bopf_t.labels)
        count = 0
        for j in range(len(real_label)):
            if predicted_label[j] == real_label[j]:
                count += 1
        acc = count / len(real_label)
        if acc > rbest_centroid:
            rbest_centroid = acc
            best_centroid = i
    s_index1 = index1[best_centroid]
    print("classify with best centroid and wd:", output_dict["bop_wd"][s_index1], 
          ", wl:", output_dict["bop_wl"][s_index1], 
          "-> cv_acc:", round(output_dict["bop_cv_acc"][s_index1], 3),
          ", acc:", round(rbest_centroid, 3))
    
#     else:
    # classify using tf-idf
    # classify using centroid
    for i in range(top_n):
        print(i, end="\r")
        s_index = index2[i]
        wd = output_dict["bop_wd"][s_index]
        wl = output_dict["bop_wl"][s_index]
        bopf_t.bop(wd, wl, verbose=False)
        test_bop_sort = sort_trim_arr(bopf_t.train_bop, output_dict["bop_feature_index2"][s_index], 
                              bopf_t.m, output_dict["bop_fea_num2"][s_index])
        predicted_label = classify2(test_bop_sort, output_dict["bop_features2"][s_index], bopf.tlabel, 
                           bopf_t.m, bopf.c, output_dict["bop_fea_num2"][s_index])
        real_label = np.array(bopf_t.labels)
        count = 0
        for j in range(len(real_label)):
            if predicted_label[j] == real_label[j]:
                count += 1
        acc = count / len(real_label)
        if acc > rbest_tf_idf:
            rbest_tf_idf = acc
            best_tf_idf = i
            
    s_index2 = index2[best_tf_idf]
    print("classify with best tf-idf and wd:", output_dict["bop_wd"][s_index2], 
          ", wl:", output_dict["bop_wl"][s_index2], 
          "-> cv_acc:", round(output_dict["bop_cv_acc2"][s_index2], 3),
          ", acc:", round(rbest_tf_idf, 3))
    return max(round(rbest_tf_idf, 3), round(rbest_centroid, 3))